In [1]:
from datasets import load_dataset, load_from_disk
DATASET_PATH = 'tmp_pony_speech'
import os

uncolumned_dataset = load_from_disk(DATASET_PATH)

# Add speaker IDs

In [2]:
from sklearn.preprocessing import LabelEncoder
import os
le = LabelEncoder()

train_dataset = uncolumned_dataset['train']
all_characters = set(train_dataset['speaker'])
le.fit(list(all_characters))

def encode_category(example):
    example['speaker_id'] = le.transform([example['speaker']])[0]
    return example

updated_dataset = train_dataset.map(encode_category,num_proc=8)

In [5]:
id_to_category = dict(enumerate(le.classes_))
import json
with open('speaker_ids_to_names.json', 'w') as f:
    json.dump(id_to_category, f)

# Remove "multiple" speakers

In [3]:
from functools import partial
speakers_exclude = {'CMC', 'Dazzlings', 'Mane 6', 'Flim Flam', 'Multiple'}

def filter_fn(ex, speakers_exclude):
    return ex['speaker'] not in speakers_exclude

updated_dataset = updated_dataset.filter(partial(
    filter_fn, speakers_exclude=speakers_exclude), num_proc=8)

In [4]:
uploaded_repo = 'therealvul/parlertts_pony_speech_ids_fixed_stage1'
updated_dataset.push_to_hub(uploaded_repo)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/64 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/therealvul/parlertts_pony_speech_ids_fixed_stage1/commit/038190277593e3b2798c7d449fbd857b4653adfd', commit_message='Upload dataset', commit_description='', oid='038190277593e3b2798c7d449fbd857b4653adfd', pr_url=None, pr_revision=None, pr_num=None)